# INSTRUCTIONS:
1. Run the first two cells
2. Run the last two cells
3. Run everything before the last two cells
4. Update the last cell `index_to_run` and run everything before the last two cells again

In [ ]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
import json
import sys
import psycopg2
import pandas as pd
import os
from datetime import datetime
from geoalchemy2.elements import WKBElement
from shapely import wkt, wkb
load_dotenv()
sys.path.append(os.path.join(os.environ['GIT_FOLDER']))
sys.path.append(os.path.join(os.environ['GIT_FOLDER'], 'cerulean_cloud', 'cloud_function_ais_analysis'))

with open(os.environ['JSON_PATH'], 'r') as f:
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json.dumps(json.load(f))
cv1_db_url = os.environ['CV1_CONNECTION_STRING']
cv2_db_url = os.environ['CV2_CONNECTION_STRING']

from cerulean_cloud.cloud_function_ais_analysis.main import *
from cerulean_cloud.cloud_function_ais_analysis.utils.constants import *
from cerulean_cloud.database_schema import *


In [ ]:

# s1 = Sentinel1Grd()
# s1.scene_id = "S1A_IW_GRDH_1SDV_20231108T051437_20231108T051502_051121_062A4D_DFBF"
# s1.geometry = WKBElement(wkb.dumps(wkt.loads("POLYGON((8.955023887201879 4.111563696931682,7.845225326041903 4.343961232447492,6.742765353681415 4.573039801014598,6.668846156780897 4.211153353413422,6.630264751712069 4.030491645888738,6.592564021309396 3.849641473483156,6.557037786766955 3.668331911990781,6.520516641414377 3.487223069292632,6.433739695202592 3.065249975902795,7.533827057230924 2.833767920845413,8.641477120289817 2.599513365462752,8.8043958596723 3.385729584075115,8.878795143375168 3.74889239584086,8.917474387970788 3.93015712327009,8.955023887201879 4.111563696931682))")))
s1.start_time = datetime.strptime(s1.scene_id[17:32], "%Y%m%dT%H%M%S")


In [ ]:
aisc = AISConstructor(s1)
print("Retrieving AIS")
aisc.retrieve_ais()
print("Building trajectories")
aisc.build_trajectories()
print("Buffering trajectories")
aisc.buffer_trajectories()


In [ ]:
print("Loading infra")
aisc.load_infra("../cerulean_cloud/cloud_function_ais_analysis/20231103_all_infrastructure_v20231103.csv")

In [ ]:
slick = Slick()
# mp = "MULTIPOLYGON(((6.904 4,6.904 4.002,6.906 4.002,6.906 4.009,6.908 4.009,6.908 4.007,6.912 4.004,6.918 4.005,6.925 4.003,6.946 4.004,6.958 4.002,6.965 4.002,6.973 4.007,6.982 4.007,6.974 4.011,6.964 4.013,6.955 4.017,6.947 4.018,6.938 4.02,6.936 4.022,6.925 4.023,6.918 4.026,6.914 4.026,6.91 4.03,6.923 4.029,6.932 4.024,6.949 4.022,6.957 4.018,6.976 4.014,6.995 4.006,7 4.006,7.002 4.008,7.002 4.005,7.006 4.003,7.002 4,6.962 4.001,6.906 3.999,6.904 4)),((7.004 4.045,7.005 4.041,7.001 4.035,7 4.039,7 4.05,7.004 4.05,7.004 4.045)),((6.991 4.045,6.99 4.045,6.99 4.047,6.993 4.047,6.991 4.045)),((6.91 4.029,6.906 4.029,6.906 4.031,6.91 4.029)),((6.904 4.033,6.906 4.033,6.904 4.033,6.903 4.03,6.901 4.03,6.899 4.033,6.899 4.037,6.904 4.036,6.904 4.033)),((6.905 4.015,6.905 4.011,6.901 4.011,6.901 4.017,6.905 4.015)),((6.901 4.001,6.904 4,6.904 3.998,6.9 3.998,6.899 4.001,6.901 4.001)))"
slick.geometry = WKBElement(wkb.dumps(wkt.loads(mp)))
slick_gdf = gpd.GeoDataFrame({'geometry': [wkb.loads(str(slick.geometry))]}, crs="4326")
slick_gdf.plot()

In [ ]:
print("Associating slicks")
source_associations = automatic_source_analysis(aisc, slick)
print("Done")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, figsize=(10, 10))


gdf1 = aisc.ais_buffered[aisc.ais_buffered['st_name'].isin(source_associations["st_name"].iloc[:1])]
gdf2 = slick_gdf
# gdf3 = source_associations[:5]
gdf4 = aisc.infra_gdf

minx, miny, maxx, maxy = gdf2.total_bounds
n = 1

width = maxx - minx
height = maxy - miny
expanded_minx = minx - width / n
expanded_maxx = maxx + width / n
expanded_miny = miny - height / n
expanded_maxy = maxy + height / n
ax.set_xlim([expanded_minx, expanded_maxx])
ax.set_ylim([expanded_miny, expanded_maxy])


gdf1.plot(ax=ax, cmap='viridis', alpha=.5)
gdf2.plot(ax=ax, color='red', alpha=1)
gdf4.plot(ax=ax, color="black")
# gdf3.plot(ax=ax, color='blue')

plt.show()


In [ ]:
print(f"{s1.scene_id}, {source_associations.iloc[0]['st_name']}")

In [ ]:
# Run once after "Run All" has failed

def fetch_data(connection, query):
    with psycopg2.connect(connection) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            return rows

pid = "','".join(['S1A_IW_GRDH_1SDV_20210409T215314_20210409T215342_037379_0467B9_03F6',
'S1A_IW_GRDH_1SDV_20210727T223312_20210727T223334_038969_049919_B9C1',
'S1A_IW_GRDH_1SDV_20210807T215206_20210807T215231_039129_049E2F_BF10',
'S1A_IW_GRDH_1SDV_20210815T114323_20210815T114348_039240_04A1EF_356C',
'S1A_IW_GRDH_1SDV_20210820T115437_20210820T115502_039313_04A47C_029B',
'S1A_IW_GRDH_1SDV_20210820T223133_20210820T223158_039319_04A4B4_90DA',
'S1A_IW_GRDH_1SDV_20210821T105041_20210821T105106_039326_04A4F4_E378',
'S1A_IW_GRDH_1SDV_20210825T223609_20210825T223638_039392_04A739_8A62',
'S1A_IW_GRDH_1SDV_20210830T224346_20210830T224415_039465_04A9AE_D4CE',
'S1A_IW_GRDH_1SDV_20210901T223134_20210901T223159_039494_04AAAE_FD16',
'S1A_IW_GRDH_1SDV_20210906T224044_20210906T224109_039567_04AD4B_0640',
'S1A_IW_GRDH_1SDV_20210920T114325_20210920T114350_039765_04B3EF_5595',
'S1A_IW_GRDH_1SDV_20210924T215143_20210924T215208_039829_04B625_1812',
'S1A_IW_GRDH_1SDV_20211006T215143_20211006T215208_040004_04BC3E_3956',
'S1A_IW_GRDH_1SDV_20211017T225016_20211017T225032_040165_04C1D2_87B8',
'S1A_IW_GRDH_1SDV_20211018T214913_20211018T214938_040179_04C256_EFB3',
'S1A_IW_GRDH_1SDV_20211018T215258_20211018T215323_040179_04C256_F17E',
'S1A_IW_GRDH_1SDV_20211019T223250_20211019T223315_040194_04C2D2_860E',
'S1A_IW_GRDH_1SDV_20211104T111709_20211104T111734_040420_04CAB8_19A1',
'S1A_IW_GRDH_1SDV_20211107T222614_20211107T222641_040471_04CC78_2EB1',
'S1A_IW_GRDH_1SDV_20211108T104207_20211108T104236_040478_04CCBD_A41D',
'S1A_IW_GRDH_1SDV_20211108T230157_20211108T230222_040486_04CD06_FE0F',
'S1A_IW_GRDH_1SDV_20211114T095415_20211114T095444_040565_04CFC2_CBFF',
'S1A_IW_GRDH_1SDV_20211116T215443_20211116T215508_040602_04D10B_3DB1',
'S1A_IW_GRDH_1SDV_20211117T101655_20211117T101720_040609_04D141_EF78',
'S1A_IW_GRDH_1SDV_20211124T223314_20211124T223339_040719_04D50A_1BD4',
'S1A_IW_GRDH_1SDV_20211217T110639_20211217T110708_041047_04E05E_7C27',
'S1A_IW_GRDH_1SDV_20211228T211038_20211228T211103_041214_04E5DE_A178',
'S1A_IW_GRDH_1SDV_20220103T215412_20220103T215441_041302_04E8DF_43A1',
'S1A_IW_GRDH_1SDV_20220112T213459_20220112T213524_041433_04ED39_0AA3',
'S1A_IW_GRDH_1SDV_20220127T111731_20220127T111756_041645_04F44B_7C58',
'S1A_IW_GRDH_1SDV_20220203T215229_20220203T215254_041754_04F7FF_8566',
'S1A_IW_GRDH_1SDV_20220204T115125_20220204T115153_041763_04F84B_A71D',
'S1A_IW_GRDH_1SDV_20220206T221429_20220206T221458_041798_04F994_0D42',
'S1A_IW_GRDH_1SDV_20220210T213919_20220210T213944_041856_04FBA1_97D7',
'S1A_IW_GRDH_1SDV_20220310T102558_20220310T102627_042257_050959_AA40',
'S1A_IW_GRDH_1SDV_20220312T223246_20220312T223311_042294_050A9E_98D6',
'S1A_IW_GRDH_1SDV_20220313T105039_20220313T105104_042301_050ADB_346D',
'S1A_IW_GRDH_1SDV_20220315T225510_20220315T225539_042338_050C1E_DB6C',
'S1A_IW_GRDH_1SDV_20220316T111755_20220316T111820_042346_050C69_F6FC',
'S1A_IW_GRDH_1SDV_20220324T223246_20220324T223311_042469_051092_7CA7',
'S1A_IW_GRDH_1SDV_20220329T223951_20220329T224016_042542_05131C_EEF5',
'S1A_IW_GRDH_1SDV_20220407T221729_20220407T221754_042673_05176E_6CDE',
'S1A_IW_GRDH_1SDV_20220414T220847_20220414T220914_042775_051AE6_9848',
'S1A_IW_GRDH_1SDV_20220415T224742_20220415T224807_042790_051B6C_6294',
'S1A_IW_GRDH_1SDV_20220417T223222_20220417T223247_042819_051C5F_4DFD',
'S1A_IW_GRDH_1SDV_20220429T115502_20220429T115527_042988_0521E1_1C93',
'S1A_IW_GRDH_1SDV_20220429T223222_20220429T223247_042994_05221A_8844',
'S1A_IW_GRDH_1SDV_20220506T222522_20220506T222547_043096_052579_798F',
'S1A_IW_GRDH_1SDV_20220525T113434_20220525T113459_043367_052DC0_86AC',
'S1A_IW_GRDH_1SDV_20220528T101655_20220528T101720_043409_052F01_B920',
'S1A_IW_GRDH_1SDV_20220626T224718_20220626T224747_043840_053BD3_7F88',
'S1A_IW_GRDH_1SDV_20220629T105713_20220629T105738_043877_053CEB_4888',
'S1A_IW_GRDH_1SDV_20220726T202115_20220726T202144_044276_0548E0_192C',
'S1A_IW_GRDH_1SDV_20220726T220116_20220726T220141_044277_0548E4_F93B',
'S1A_IW_GRDH_1SDV_20220807T220027_20220807T220052_044452_054DFF_EF88',
'S1A_IW_GRDH_1SDV_20220807T220117_20220807T220142_044452_054DFF_9CA4',
'S1A_IW_GRDH_1SDV_20220820T224050_20220820T224115_044642_05544F_7001',
'S1A_IW_GRDH_1SDV_20220822T114150_20220822T114215_044664_055500_6576',
'S1A_IW_GRDH_1SDV_20220831T220053_20220831T220118_044802_0559BB_522A',
'S1A_IW_GRDH_1SDV_20220920T223345_20220920T223410_045094_05637D_7BB5',
'S1A_IW_GRDH_1SDV_20220929T234119_20220929T234148_045226_0567E3_2A50',
'S1A_IW_GRDH_1SDV_20221004T221439_20221004T221508_045298_056A61_A471',
'S1A_IW_GRDH_1SDV_20221006T111650_20221006T111715_045320_056B1C_2FB0',
'S1A_IW_GRDH_1SDV_20221012T224353_20221012T224422_045415_056E4A_AA40',
'S1A_IW_GRDH_1SDV_20221015T105534_20221015T105603_045452_056F76_08FB',
'S1A_IW_GRDH_1SDV_20221016T113416_20221016T113441_045467_056FE9_EDC5',
'S1A_IW_GRDH_1SDV_20221020T214428_20221020T214453_045531_05717D_30F1',
'S1A_IW_GRDH_1SDV_20221031T223932_20221031T224001_045692_0576F4_B4C5',
'S1A_IW_GRDH_1SDV_20221031T224026_20221031T224051_045692_0576F4_AC86',
'S1A_IW_GRDH_1SDV_20221106T215214_20221106T215239_045779_0579DB_F7F1',
'S1A_IW_GRDH_1SDV_20221115T230203_20221115T230228_045911_057E4B_C7C2',
'S1A_IW_GRDH_1SDV_20221119T115416_20221119T115445_045963_058000_2CF9',
'S1A_IW_GRDH_1SDV_20221123T111715_20221123T111740_046020_058209_9674',
'S1A_IW_GRDH_1SDV_20221203T113555_20221203T113620_046167_0586F7_57D2',
'S1A_IW_GRDH_1SDV_20221208T222529_20221208T222554_046246_0589A8_7F57',
'S1A_IW_GRDH_1SDV_20221217T111739_20221217T111804_046370_058DF6_EB88',
'S1A_IW_GRDH_1SDV_20230109T225636_20230109T225701_046713_059983_2EC4',
'S1A_IW_GRDH_1SDV_20230110T111532_20230110T111557_046720_0599CC_D0F9',
'S1A_IW_GRDH_1SDV_20230203T111557_20230203T111622_047070_05A589_1871',
'S1A_IW_GRDH_1SDV_20230203T112007_20230203T112032_047071_05A589_691B',
'S1A_IW_GRDH_1SDV_20230215T111826_20230215T111851_047246_05AB64_3780',
'S1A_IW_GRDH_1SDV_20230227T111942_20230227T112007_047421_05B161_240D',
'S1A_IW_GRDH_1SDV_20230304T112807_20230304T112832_047494_05B3D5_FA08',
'S1A_IW_GRDH_1SDV_20230307T115041_20230307T115106_047538_05B550_E3EC',
'S1A_IW_GRDH_1SDV_20230311T111736_20230311T111801_047595_05B74B_6351',
'S1A_IW_GRDH_1SDV_20230319T115131_20230319T115159_047713_05BB3B_F9EB',
'S1A_IW_GRDH_1SDV_20230323T111621_20230323T111646_047770_05BD30_7FF2',
'S1A_IW_GRDH_1SDV_20230402T203727_20230402T203756_047922_05C245_5FCF',
'S1A_IW_GRDH_1SDV_20230420T104210_20230420T104238_048178_05CAE4_EC76',
'S1A_IW_GRDH_1SDV_20230420T230225_20230420T230258_048186_05CB29_85A6',
'S1A_IW_GRDH_1SDV_20230518T223139_20230518T223204_048594_05D839_0A19',
'S1B_IW_GRDH_1SDV_20210713T111921_20210713T111950_027775_035088_7174',
'S1B_IW_GRDH_1SDV_20210804T095630_20210804T095655_028095_035A02_2970',
'S1B_IW_GRDH_1SDV_20210824T102537_20210824T102602_028386_036318_EECA',
'S1B_IW_GRDH_1SDV_20210826T100855_20210826T100924_028415_0363F1_D947',
'S1B_IW_GRDH_1SDV_20211207T100514_20211207T100539_029918_03924D_CD13',
'S1B_IW_GRDH_1SSV_20211112T105804_20211112T105832_029553_0386F3_BBD7',])

cv1_sql = f"""
SELECT 
DISTINCT ON (grd__pid)
    a.geometry AS posi_poly__poly,
    st_astext(d.geometry) AS grd__geometry,
    d.pid AS grd__pid,
    d.starttime AS grd__starttime,
    e.class_int AS slick__class_int,
    e.id AS slick__id,
    a.id AS posi_poly__id
FROM posi_poly a
    JOIN inference c ON c.id = a.inference__id
    JOIN grd d ON d.id = c.grd__id
    JOIN slick e ON e.id = a.slick__id
WHERE true
    AND d.pid in ('{pid}')
ORDER BY grd__pid, (st_perimeter(a.geometry) * st_perimeter(a.geometry) / st_area(a.geometry)) DESC;
"""

data = fetch_data(cv1_db_url, cv1_sql)
df = pd.DataFrame(data, columns=['geometry', 'grd__geometry', "grd__pid", "grd__starttime", "slick__class_int", "slick__id", "posi_poly__id"])
df["geometry"] = wkb.loads(df["geometry"], hex=True)
gdf=gpd.GeoDataFrame(df, geometry="geometry")
gdf.iloc[0]["geometry"]

In [ ]:
# Run the following for each value of index_to_run

index_to_run = 1

s = gdf.iloc[index_to_run]
s1 = Sentinel1Grd()
s1.scene_id = s["grd__pid"]
s1.geometry = WKBElement(wkb.dumps(wkt.loads(s["grd__geometry"])))
mp = s["geometry"].wkt